<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Recording-&amp;-ASR" data-toc-modified-id="Recording-&amp;-ASR-1">Recording &amp; ASR</a></span></li></ul></div>

In [22]:
!pip install python-Levenshtein

In [23]:
import os
print(os.environ['PATH'])

/home/kwangho/kaldi/src/bin:/home/kwangho/kaldi/src/chainbin:/home/kwangho/kaldi/src/featbin:/home/kwangho/kaldi/src/fgmmbin:/home/kwangho/kaldi/src/fstbin:/home/kwangho/kaldi/src/gmmbin:/home/kwangho/kaldi/src/ivectorbin:/home/kwangho/kaldi/src/kwsbin:/home/kwangho/kaldi/src/latbin:/home/kwangho/kaldi/src/lmbin:/home/kwangho/kaldi/src/nnet2bin:/home/kwangho/kaldi/src/nnet3bin:/home/kwangho/kaldi/src/nnetbin:/home/kwangho/kaldi/src/online2bin:/home/kwangho/kaldi/src/onlinebin:/home/kwangho/kaldi/src/rnnlmbin:/home/kwangho/kaldi/src/sgmm2bin:/home/kwangho/kaldi/src/sgmmbin:/home/kwangho/kaldi/src/tfrnnlmbin:/home/kwangho/kaldi/src/cudadecoderbin:/home/kwangho/kaldi/egs/zeroth_korean/s5/utils:/home/kwangho/kaldi/tools/openfst/bin:/home/kwangho/kaldi/egs/zeroth_korean/s5:/home/kwangho/anaconda3/envs/kaldi/bin:/home/kwangho/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/local/kaldi/tools/morfessor/scripts


In [24]:
#!/usr/bin/env python

from __future__ import print_function

# word confidence
from kaldi.lat.sausages import MinimumBayesRisk, MinimumBayesRiskOptions

# sent confidence
from kaldi.lat.functions import sentence_level_confidence

# alignement
from kaldi.alignment import NnetAligner
from kaldi.fstext import SymbolTable
from kaldi.lat.align import WordBoundaryInfoNewOpts, WordBoundaryInfo
from kaldi.nnet3 import NnetSimpleComputationOptions
from kaldi.util.table import SequentialMatrixReader

# decoder
from kaldi.asr import NnetLatticeFasterOnlineRecognizer
from kaldi.decoder import LatticeFasterDecoderOptions
from kaldi.nnet3 import NnetSimpleLoopedComputationOptions
from kaldi.online2 import (OnlineEndpointConfig,
                           OnlineIvectorExtractorAdaptationState,
                           OnlineNnetFeaturePipelineConfig,
                           OnlineNnetFeaturePipelineInfo,
                           OnlineNnetFeaturePipeline,
                           OnlineSilenceWeighting)

from kaldi.util.options import ParseOptions
from kaldi.util.table import SequentialWaveReader

from kaldi.fstext import SymbolTable, shortestpath, indices_to_symbols
from kaldi.fstext.utils import get_linear_symbol_sequence


In [16]:
chunk_size = 1440

# Define online feature pipeline
feat_opts = OnlineNnetFeaturePipelineConfig()
endpoint_opts = OnlineEndpointConfig()
po = ParseOptions("")
feat_opts.register(po)
endpoint_opts.register(po)
po.read_config_file("./chain/tdnn1a_sp_online/conf/online.conf")
feat_info = OnlineNnetFeaturePipelineInfo.from_config(feat_opts)

# Construct recognizer
decoder_opts = LatticeFasterDecoderOptions()
decoder_opts.beam = 15.0
decoder_opts.max_active = 7000
decoder_opts.min_active = 200
decoder_opts.lattice_beam = 6.0

decodable_opts = NnetSimpleLoopedComputationOptions()
decodable_opts.acoustic_scale = 1.0
decodable_opts.frame_subsampling_factor = 3
decodable_opts.frames_per_chunk = 20
decodable_opts.extra_left_context_initial = 0

asr = NnetLatticeFasterOnlineRecognizer.from_files(
    "./chain/tdnn1a_sp_online/final.mdl", 
    "./chain/tree_a/graph_tgsmall/HCLG.fst", 
    "./chain/tree_a/graph_tgsmall/words.txt",
    decoder_opts=decoder_opts,
    decodable_opts=decodable_opts,
    endpoint_opts=endpoint_opts)

print("ASR Model LOAD Complete")

ASR Model LOAD Complete


In [25]:
# Construct aligner
decodable_opts = NnetSimpleComputationOptions()
decodable_opts.acoustic_scale = 1.0
decodable_opts.frame_subsampling_factor = 3

aligner = NnetAligner.from_files(
    "./chain/tdnn1a_sp_online/final.mdl", 
    "./chain/tree_a/tree", 
    "../data/lang_test_tgsmall/L.fst", 
    "./chain/tree_a/graph_tgsmall/words.txt",
    "./chain/tree_a/graph_tgsmall/phones/disambig.int", 
    decodable_opts=decodable_opts)

phones = SymbolTable.read_text("./chain/tree_a/graph_tgsmall/phones.txt")
wb_info = WordBoundaryInfo.from_file(WordBoundaryInfoNewOpts(),
                                     "./chain/tree_a/graph_tgsmall/phones/word_boundary.int")


print("ASR Aligner LOAD Complete")

ASR Aligner LOAD Complete


In [6]:
!locale

LANG=ko_KR.UTF-8
LANGUAGE=
LC_CTYPE="ko_KR.UTF-8"
LC_NUMERIC="ko_KR.UTF-8"
LC_TIME="ko_KR.UTF-8"
LC_COLLATE="ko_KR.UTF-8"
LC_MONETARY="ko_KR.UTF-8"
LC_MESSAGES="ko_KR.UTF-8"
LC_PAPER="ko_KR.UTF-8"
LC_NAME="ko_KR.UTF-8"
LC_ADDRESS="ko_KR.UTF-8"
LC_TELEPHONE="ko_KR.UTF-8"
LC_MEASUREMENT="ko_KR.UTF-8"
LC_IDENTIFICATION="ko_KR.UTF-8"
LC_ALL=ko_KR.UTF-8


In [7]:
ref = {}
with open("text", 'r') as f:
    for line in f:
        token = line.split()
        key = token[0]
        sent = ' '.join(token[1:])
        ref[key] = sent
    
# print(ref)
print("")

wav_fnames = {}
with open("wav.scp", 'r') as f:
    for line in f:
        token = line.split()
        key = token[0]
        fname = token[-2]
        wav_fnames[key] = fname
    
# print(wav_fnames)
print("")

In [18]:
import Levenshtein as Lev

class TranscriptionError(object):
    def __init__(self, word_errors=None, char_errors=None, tokens=0, chars=0):
        self.word_errors = word_errors if word_errors is not None else {"delete": 0, "insert": 0, "replace": 0}
        self.char_errors = char_errors if char_errors is not None else {"delete": 0, "insert": 0, "replace": 0}
        self.tokens = tokens
        self.chars = chars

    @property
    def wer(self):
        return 100 * sum(self.word_errors.values()) / max(1, self.tokens)

    @property
    def cer(self):
        return 100 * sum(self.char_errors.values()) / max(1, self.chars)

    @classmethod
    def calculate(cls, hypothesis, reference):
        word_errors = TranscriptionError._get_word_errors(hypothesis, reference)
        char_errors = TranscriptionError._get_char_errors(hypothesis, reference)

        return cls(word_errors, char_errors, len(reference.split()), len(reference))

    @staticmethod
    def _get_word_errors(s1, s2):
        b = set(s1.split() + s2.split())
        word2char = dict(zip(b, range(len(b))))
        w1 = [chr(word2char[w]) for w in s1.split()]
        w2 = [chr(word2char[w]) for w in s2.split()]

        ops = Lev.editops(''.join(w1), ''.join(w2))
        errors = {"delete": 0, "insert": 0, "replace": 0}
        for x in ops:
            errors[x[0]] += 1
        return errors

    @staticmethod
    def _get_char_errors(s1, s2):
        s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')
        ops = Lev.editops(s1, s2)
        errors = {"delete": 0, "insert": 0, "replace": 0}
        for x in ops:
            errors[x[0]] += 1
        return errors

In [10]:
import time
import librosa

# Decode (whole utterance)
for idx, (key, wav) in enumerate(SequentialWaveReader("scp:wav.scp")):
    start = time.time()
    feat_pipeline = OnlineNnetFeaturePipeline(feat_info)
    asr.set_input_pipeline(feat_pipeline)
    feat_pipeline.accept_waveform(wav.samp_freq, wav.data()[0])
    feat_pipeline.input_finished()
    out = asr.decode()
    # print(out)
    end = time.time()
    
    print("#"*70)
    print("IDX: ", (idx+1), "// ID : ", key)
    print("REF: ", ref[key], flush=True)
    print("HYP: ", out["text"], flush=True)
    
    te = TranscriptionError.calculate(out["text"], ref[key])
    print("WER: ", "{:.2f}(%) \t".format(te.wer), 
          "I: {insert:02d}, D: {delete:02d}, S: {substitute:02d}".format(
              insert=te.word_errors['insert'], 
              delete=te.word_errors['delete'],
              substitute=te.word_errors['replace']))
    print("CER: ", "{:.2f}(%) \t".format(te.cer), 
          "I: {insert:02d}, D: {delete:02d}, S: {substitute:02d}".format(
              insert=te.char_errors['insert'], 
              delete=te.char_errors['delete'],
              substitute=te.char_errors['replace']))
    
    
    y, sr = librosa.load(wav_fnames[key])
    dur = len(y) / sr
    print("Duration: {:.2f}(sec.), Response Time: {:.2f}(sec.), RTF: {:.2f}".format(
            dur, end-start, (end-start)/dur))
    
    
    word_alignment = aligner.to_word_alignment(out["best_path"], wb_info)
    for idx, (word, start_frame, offset) in enumerate(word_alignment):
        start_frame *= decodable_opts.frame_subsampling_factor
        offset *= decodable_opts.frame_subsampling_factor
        print("W: {:12s} \t start_frame: \t {:4d} \t end_frame: \t {:4d}".format(word, start_frame, (start_frame+offset)))
    
    # print("{:02d} : ".format(idx), "alignment : ", key, word_alignment, flush=True)

    
    print("")
    

######################################################################
IDX:  1 // ID :  104_003_0019
REF:  지난해 삼 월 김 전 장관의 동료 인 장동 련 홍익대 교수가 민간 자문단 장으로 위촉 되면서 본격적인 공모 와 개발 작업에 들어갔다
HYP:  지난해 [3] 월 진전 장관의 동료 인 장동영 홍익대 교수가 민간 자문단 장으로 위축 되면서 본격적인 공모 와 개발 작업에 들어갔다
WER:  26.09(%) 	 I: 02, D: 00, S: 04
CER:  8.22(%) 	 I: 00, D: 02, S: 04
Duration: 10.93(sec.), Response Time: 2.52(sec.), RTF: 0.23
W: <eps>        	 start_frame: 	    0 	 end_frame: 	   75
W: 지난해          	 start_frame: 	   75 	 end_frame: 	  117
W: [3]          	 start_frame: 	  117 	 end_frame: 	  135
W: 월            	 start_frame: 	  135 	 end_frame: 	  156
W: 진전           	 start_frame: 	  156 	 end_frame: 	  201
W: 장관의          	 start_frame: 	  201 	 end_frame: 	  243
W: 동료           	 start_frame: 	  243 	 end_frame: 	  282
W: 인            	 start_frame: 	  282 	 end_frame: 	  303
W: 장동영          	 start_frame: 	  303 	 end_frame: 	  369
W: 홍익대          	 start_frame: 	  369 	 end_frame: 	  420
W: 교수가          	 start_fr

# Recording & ASR

In [66]:
src_fname = "./wav/test.wav"
dest_fname = "./wav/test.SR_16000.CH_1.BI_16.wav"

In [67]:
!ffmpeg -y -i $src_fname -acodec pcm_s16le -ac 1 -ar 16000 $dest_fname

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [68]:
!file $dest_fname

./wav/test.SR_16000.CH_1.BI_16.wav: RIFF (little-endian) data, WAVE audio, Microsoft PCM, 16 bit, mono 16000 Hz


In [69]:
import IPython.display as ipd
ipd.Audio(dest_fname) # load a local WAV file

In [70]:
with open("wav.kwangho.scp", "w") as f:
    f.write("{} {}".format(1, dest_fname))

In [71]:
import time
import librosa

# Decode (whole utterance)
for idx, (key, wav) in enumerate(SequentialWaveReader("scp:wav.kwangho.scp")):
    start = time.time()
    feat_pipeline = OnlineNnetFeaturePipeline(feat_info)
    asr.set_input_pipeline(feat_pipeline)
    feat_pipeline.accept_waveform(wav.samp_freq, wav.data()[0])
    feat_pipeline.input_finished()
    out = asr.decode()
    # print(out)
    end = time.time()
    
    print("#"*70)
    print("IDX: ", (idx+1), "// ID : ", key)
    print("HYP: ", out["text"], flush=True)
    
    y, sr = librosa.load(dest_fname, sr=16000)
    dur = len(y) / sr
    print("Duration: {:.2f}(sec.), Response Time: {:.2f}(sec.), RTF: {:.2f}".format(
            dur, end-start, (end-start)/dur))
    
    
    word_alignment = aligner.to_word_alignment(out["best_path"], wb_info)
    for idx, (word, start_frame, offset) in enumerate(word_alignment):
        start_frame *= decodable_opts.frame_subsampling_factor
        offset *= decodable_opts.frame_subsampling_factor
        print("W: {:12s} \t start_frame: \t {:4d} \t end_frame: \t {:4d}".format(word, start_frame, (start_frame+offset)))
    
    # print("{:02d} : ".format(idx), "alignment : ", key, word_alignment, flush=True)

    
    print("")

######################################################################
IDX:  1 // ID :  1
HYP:  지금 은 공덕 에서 교육 중입니다
Duration: 6.07(sec.), Response Time: 0.97(sec.), RTF: 0.16
W: <eps>        	 start_frame: 	    0 	 end_frame: 	   99
W: 지금           	 start_frame: 	   99 	 end_frame: 	  138
W: 은            	 start_frame: 	  138 	 end_frame: 	  153
W: 공덕           	 start_frame: 	  153 	 end_frame: 	  201
W: 에서           	 start_frame: 	  201 	 end_frame: 	  264
W: <eps>        	 start_frame: 	  264 	 end_frame: 	  279
W: 교육           	 start_frame: 	  279 	 end_frame: 	  315
W: 중입니다         	 start_frame: 	  315 	 end_frame: 	  378
W: <eps>        	 start_frame: 	  378 	 end_frame: 	  606



In [73]:
import IPython.display as ipd
ipd.Audio(dest_fname) # load a local WAV file